In [ ]:
# !pip install -U "mlfoundry<0.5.0"

In [ ]:
!mlfoundry login --tracking_uri "https://app.truefoundry.com"

MlFoundry CLI
Please get your API key from https://app.truefoundry.com/settings
Paste your API key and hit enter:
Writing API key at /root/.mlfoundry/credentials.netrc


In [ ]:
from datetime import datetime
import pickle
import pandas as pd 
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

train_data_path = "train.csv"
test_data_path = "test.csv"


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
df=pd.read_csv(train_data_path)
test=pd.read_csv(test_data_path)
text=df['tweet'].values.tolist()
text_test=test['tweet'].values.tolist()
text+=text_test

In [ ]:
stopword=nltk.corpus.stopwords.words('english')
stopword.remove('not')
for index,text_ in enumerate(text):
    text_=re.sub(r'@[\w]*','',text_) #Removing Twitter Handles (@user)
    text_=re.sub(r'http/S+','',text_) #Removing urls from text 
    text_=re.sub(r'[^A-Za-z#]',' ',text_) #Removing Punctuations, Numbers, and Special Characters
    text_=" ".join(i.lower() for i in text_.split() if i.lower() not in stopword) #Removing stopword
    text[index]=text_


In [ ]:
#Stemming the word
pt=PorterStemmer()
wordnet=WordNetLemmatizer()
for index,text_ in enumerate(text):
    text_=" ".join(pt.stem(i) for i in text_.split())
    text_=" ".join(wordnet.lemmatize(i) for i in text_.split())  
    text[index]=text_

df['preprocess_tweet']=text[:len(df)]
df['length_tweet']=df['preprocess_tweet'].str.len()
test['preprocess_tweet']=text[len(df):]

In [ ]:
train=df.copy()
train.drop(columns=['id','tweet','preprocess_tweet'],inplace=True)

In [ ]:
train.shape, test.shape

((31962, 2), (17197, 3))

In [ ]:
bow=CountVectorizer(max_features=1000)
bow.fit(df['preprocess_tweet'])
bow_df=bow.transform(df['preprocess_tweet']).toarray()
print('feature name==',bow.get_feature_names_out()[:10])
print('number of uniqe words',bow_df.shape[1])
print('shape',bow_df.shape)

feature name== ['abl' 'absolut' 'accept' 'account' 'act' 'action' 'actor' 'actual' 'ad'
 'adapt']
number of uniqe words 1000
shape (31962, 1000)


In [ ]:
bow_train=pd.DataFrame(bow_df)
bow_train['length_tweet']=df['length_tweet']
bow_train['label']=df['label']

major_class_0,major_class_1=bow_train.label.value_counts()
df_major=bow_train[bow_train['label']==0]
df_minor=bow_train[bow_train['label']==1]
df_minor_upsampled = resample(df_minor, 
                                replace=True,     # sample with replacement
                                n_samples=major_class_0)
df_bow_upsampled = pd.concat([df_major, df_minor_upsampled])

x=df_bow_upsampled.iloc[:,0:-1]
y=df_bow_upsampled['label']
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2)
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (47552, 1001)
y_train (47552,)
X_test (11888, 1001)
y_test (11888,)


In [ ]:
def f1_score_(y_proba,y_test):
  proba = y_proba[:,1] >= 0.3
  proba = proba.astype(np.int) 
  return f1_score( proba,y_test) 

In [ ]:
k=[3]
accuracy_train=[]
accuracy_test=[]
metadata = {}

for i in tqdm(k):
    model=KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train,y_train)
    y_pred=model.predict(X_train)
    acc=accuracy_score(y_pred,y_train)

    print('for k=',i,'Accuracy Score',acc)

    accuracy_train.append(acc)
    y_proba=model.predict_proba(X_train)
    f1_scor_train=f1_score_(y_proba,y_train)

    print('for k=',i,'f1 score ',f1_scor_train)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


for k= 3 Accuracy Score 0.916239064602961


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 1/1 [05:45<00:00, 345.90s/it]

for k= 3 f1 score  0.8518558326167956


In [ ]:
if X_test is not None and y_test is not None:
    for i in tqdm(k):
        model=KNeighborsClassifier(n_neighbors=i)
        model.fit(X_train,y_train)
        y_pred=model.predict(X_test)
        acc=accuracy_score(y_pred,y_test)

        print('for k=',i,'Accuracy Score',acc)

        accuracy_test.append(acc)
        y_proba=model.predict_proba(X_test)
        f1_scor_test=f1_score_(y_proba,y_test)

        print('for k=',i,'f1 score ',f1_scor_test)

metadata["accuracy_train"] = accuracy_train
metadata["accuracy_test"] = accuracy_test
metadata["f1_scor_train"] = f1_scor_train
metadata["f1_scor_test"] = f1_scor_test
metadata["y_pred"] = y_pred


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


for k= 3 Accuracy Score 0.8595222072678331


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 1/1 [01:23<00:00, 83.91s/it]

for k= 3 f1 score  0.8238648216396635


In [ ]:
## Deploy model 

In [ ]:
import mlfoundry
run = mlfoundry.get_client().create_run(project_name="racial-slur-detector", run_name=f"train-{datetime.now().strftime('%m-%d-%Y')}")
run.log_params(model.get_params())

with open('vectorizer.pickle', 'wb') as fout:
    pickle.dump((bow), fout)
    
run.log_metrics({
    'train/accuracy_score': metadata["accuracy_train"][-1],
    'train/f1': metadata["f1_scor_train"],
    'test/accuracy_score': metadata["accuracy_test"][-1],
    'test/f1': metadata["f1_scor_test"]})

# run.log_dataset(
#     dataset_name='train',
#     features=X_train,
#     actuals=y_train)

# run.log_dataset(
#     dataset_name='test',
#     features=X_test,
#     predictions=metadata["y_pred"],
#     actuals=y_test,
# )

model_version = run.log_model(
    name="KNN-classifier",
    model=model,
    framework="sklearn",
    description="model trained for racial slur detection",
)
model_artifact = run.log_artifact(local_path="vectorizer.pickle", artifact_path="my-artifacts")

print(f"Logged model: {model_version.fqn}")


[mlfoundry] 2022-11-06T18:27:07+0000 INFO Welcome user! You are logged in as Urja-Jobanputra
Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/365/run/9401703fcedb43ff80580a43d6256c37/
[mlfoundry] 2022-11-06T18:27:08+0000 WARNING failed to log git info because git repository is not present
[mlfoundry] 2022-11-06T18:27:08+0000 INFO Run 'truefoundry/Urja-Jobanputra/twitter-sentiment/train-11-06-2022-20' has started.
[mlfoundry] 2022-11-06T18:27:09+0000 INFO Parameters logged successfully
[mlfoundry] 2022-11-06T18:27:09+0000 INFO Metrics logged successfully
[mlfoundry] 2022-11-06T18:27:09+0000 INFO Logging model and additional files, this might take a while ...
[mlfoundry] 2022-11-06T18:27:09+0000 INFO Serializing model files to model version contents
[mlfoundry] 2022-11-06T18:27:15+0000 INFO Packaging and uploading files to remote ...
[mlfoundry] 2022-11-06T18:27:34+0000 INFO Logged model successfully with fqn 'model:truefoundry/Urja-Jobanputra/twitter-sentiment/KN

In [ ]:
## Creating FastAPI inference endpoint 

In [ ]:
%%writefile main.py
from fastapi import FastAPI
from pydantic import BaseModel
import mlfoundry as mlf
import pandas as pd
import yaml
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import pickle

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

app = FastAPI(docs_url="/")


@app.get("/")
async def root():
    return {"message": "Welcome to Racial Slur Detector inference"}

class SentimentAnalysis(BaseModel):
    tweet: str
 
with open("infer.yaml", "r") as stream:
    try:
        env_vars = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# Load the model from MLFoundry by proving the MODEL_FQN
client = mlf.get_client(api_key=env_vars['components'][0]['env']['MLF_API_KEY'],tracking_uri=env_vars['components'][0]['env']['MLF_HOST'])
model_version = client.get_model(env_vars['components'][0]['env']['MODEL_FQN'])
model = model_version.load()

def preprocessed_tweet(test):
    text = test['tweet'].values.tolist()

    stopword=nltk.corpus.stopwords.words('english')
    stopword.remove('not')
    for index,text_ in enumerate(text):
        text_=re.sub(r'@[\w]*','',text_) #Removing Twitter Handles (@user)
        text_=re.sub(r'http/S+','',text_) #Removing urls from text 
        text_=re.sub(r'[^A-Za-z#]',' ',text_) #Removing Punctuations, Numbers, and Special Characters
        text_=" ".join(i.lower() for i in text_.split() if i.lower() not in stopword) #Removing stopword
        text[index]=text_

    #Stemming the word
    pt=PorterStemmer()
    wordnet=WordNetLemmatizer()
    for index,text_ in enumerate(text):
        text_=" ".join(pt.stem(i) for i in text_.split())
        text_=" ".join(wordnet.lemmatize(i) for i in text_.split())  
        text[index]=text_

    df_test = pd.DataFrame()
    df_test['preprocess_tweet'] = text

    run = client.get_run(env_vars['components'][0]['env']['RUN_ID'])

    vectorizer_path = run.download_artifact(path="my-artifacts/vectorizer.pickle")
    with open(vectorizer_path, 'rb') as f:
        bow = pickle.load(f)

    bow_df=bow.transform(df_test['preprocess_tweet']).toarray()
    print('feature name==',bow.get_feature_names_out()[:10])
    print('number of uniqe words',bow_df.shape[1])
    print('shape',bow_df.shape)
    bow_train=pd.DataFrame(bow_df)
    bow_train['length_tweet']=df_test['preprocess_tweet'].str.len()
    return bow_train


@app.post("/predict")
def predict(tweet):
    predict = [tweet]
    test = pd.DataFrame(data=[predict],columns=['tweet'])
    to_predict = preprocessed_tweet(test)

    print(to_predict.shape)
    prediction = model.predict(to_predict)
    return {'sentiment' : prediction.tolist()[0]}

out = predict("It's a good day")
print(out)

Writing main.py


In [ ]:
%%writefile requirements.txt
fastapi==0.82.0
uvicorn==0.18.3
pydantic==1.10.2
mlfoundry>=0.4.3,<0.5
pandas==1.4.4
nltk
sklearn
numpy

Writing requirements.txt


In [ ]:
## Deploying the FastAPI endpoint as a service to ServiceFoundry


In [ ]:
!pip install servicefoundry
!sfy logout
!sfy use server https://app.truefoundry.com
!sfy login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 3.4 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 56 kB 3.8 MB/s 
     |████████████████████████████████| 78 kB 6.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 96 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 54.0 MB/s 
     |████████████████████████████████| 59 kB 5.5 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 57 kB 3.5 MB/s 
     |████████████████████████████████| 237 kB 52.8 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 80 kB 8.6 MB/s 
     |████████████████████████████████| 52 

You are already logged out
Login Code: G64XCN
Waiting for authentication. Go to the following url to complete the authentication: 
https://app.truefoundry.com/authorize/device?userCode=G64XCN
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://app.truefoundry.com/authorize/device?userCode=G64XCN'
Login Successful!
Session stored at /root/.truefoundry/sessions.json
Setting cluster 'tfy-cluster-euwe1' as default
You are logged in as 'Urja-Jobanputra' with email 'urjajobanputra657@gmail.com'



In [ ]:
%%writefile main.py
name: sentiment-analysis-inf
components:
- name: sentiment-analysis-inf
  type: job
  image:
    type: build
    build_source:
      type: local
    build_spec:
      type: tfy-python-buildpack
      command: uvicorn infer_realtime:app --port 8000 --host 0.0.0.0
  env:
    MLF_HOST: https://app.truefoundry.com
    MLF_API_KEY: 'djE6dHJ1ZWZvdW5kcnk6VXJqYS1Kb2JhbnB1dHJhOjUwYjQ4NQ==' # Get the API_KEY from the settings of truefoundry account
    MODEL_FQN: <paste you model version fqn here>
    WORKSPACE_FQN: <paste you workspace fqn here> # Add the WORKSPACE_FQN
    RUN_ID: <paste you run id here>
  ports:
      - port: 8000

In [ ]:
import yaml
from servicefoundry import Build, PythonBuild, Service, Resources

with open("infer.yaml", "r") as stream:
    try:
        env_vars = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# Since we are using FastAPI service to infer the model we'll use the Service() method 
service = Service(
    name=env_vars['name'],
    image=Build(
        build_spec=PythonBuild(
            command=env_vars['components'][0]['image']['build_spec']['command'],
        ),
    ),
    ports=[{"port": 8000}],
    resources=Resources(memory_limit=1500, memory_request=1000),
)
service.deploy(workspace_fqn=env_vars['components'][0]['env']['WORKSPACE_FQN'])

In [ ]:
# Creating a demo webapp 

In [ ]:
 !mkdir -p /content/demo/
%cd /content/demo/

In [ ]:
%%writefile demo.py
from operator import concat
import streamlit as st
import requests
import yaml
import base64

def add_bg_from_local(image_file):
    with open(image_file, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
    st.markdown(
    f"""
    <style>
    .stApp {{
        background-image: url(data:image/{"jpeg"};base64,{encoded_string.decode()});
        background-size: cover
    }}
    </style>
    """,
    unsafe_allow_html=True
    )

with open("demo.yaml", "r") as stream:
    try:
        env_vars = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

def fetch(session, url):
    try:
        result = session.get(url)
        return result.json()
    except Exception:
        return {}

def main():
    request_url = env_vars['components'][0]['env']['INFER_URL']
    st.set_page_config(page_title="Racial Slur Detector")
    add_bg_from_local('image2.jpeg')
    st.title("Racial Slur Detector")
    st.header('Welcome to Racial Slur Detector!')
    st.write('This is a sample app that demonstrates the prowess of ServiceFoundry ML model deployment.🚀')
    st.write('Visit the [Github](https://github.com/urja0901/racial-slur-detector) repo for code or [Google Colab](https://colab.research.google.com/drive/1mWhYBiVnduQHrUqazu-fwzWze7zjGt-W?usp=sharing) notebook for a quick start.')
    with st.form("my_form"):
        
        sentiment_text = st.text_input('Sentiment Text',value="It's a good day!")

        features = {
                "tweet": sentiment_text
            }
            
        submitted = st.form_submit_button("Submit")
        if submitted:
            data = requests.post(url=concat(request_url, "/predict"), params=features).json()
            if data:
                if data["sentiment"] == 0 :
                    return_val = "positive"
                else : 
                    return_val = "negative (racist/sexist)"
                st.metric(label="sentiment",value=data['sentiment'])
            else:
                st.error("Error")

    st.image('image1.jpeg', use_column_width='always')

if __name__ == '__main__':
    main()

In [ ]:
%%writefile requirements.txt
pandas==1.4.4

# for experiment tracking and model registry
mlfoundry>=0.4.3,<0.5

# for deploying our job deployments
servicefoundry>=0.1.97,<0.2.0

streamlit>=1.13.0

In [ ]:
import argparse
import logging

from servicefoundry import Build, PythonBuild, Resources, Service

logging.basicConfig(level=logging.INFO)

# parsing the input arguments
parser = argparse.ArgumentParser()
parser.add_argument(
    "--workspace_fqn",
    type=str,
    required=True,
    help="fqn of workspace where you want to deploy",
)

parser.add_argument(
    "--inference_server_url",
    type=str,
    required=True,
    help="end point of the trained model that would be used for inference",
)

args = parser.parse_args()

# creating a service object and defining all the configurations
service = Service(
    name="sentiment-analysis-demo",
    image=Build(
        build_spec=PythonBuild(
            command="streamlit run demo.py",
            python_version="3.9",
        ),
    ),
    env={
        # These will automatically map the secret value to the environment variable.
        "MLF_HOST": "https://app.truefoundry.com",
        "INFERENCE_SERVER_URL": args.inference_server_url,
        "MLF_API_KEY": "djE6dHJ1ZWZvdW5kcnk6VXJqYS1Kb2JhbnB1dHJhOjUwYjQ4NQ==",
    },
    ports=[{"port": 8501}], #In public cloud deployment TrueFoundry exposes port 8501
    resources=Resources(
        cpu_request=0.5, cpu_limit=.5, memory_limit=2500, memory_request=1500
    ),
)
service.deploy(workspace_fqn=args.workspace_fqn)